In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [16]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("dataset/aihub/train.csv", sep=',', encoding='cp949')
test = pd.read_csv("dataset/aihub/validation.csv", sep=',', encoding='cp949')

print(train.shape)
print(test.shape)

(228468, 3)
(28555, 3)


In [17]:
# 리뷰 문장 추출
sentences = train['sentence']
sentences[:10]

0                            아내가 드디어 출산하게 되어서 정말 신이 나.
1              당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2              고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3        재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4                         빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.
5    이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를...
6                        사십 대에 접어들면서 머리카락이 많이 빠져 고민이야.
7                                       이제 돈이라면 지긋지긋해.
8               친구들이 나를 괴롭혀. 부모님과 선생님께 얘기했는데도 믿어주지 않아.
9                                         친구 때문에 눈물 나.
Name: sentence, dtype: object

In [19]:
del train['id']
del test['id']

In [22]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 아내가 드디어 출산하게 되어서 정말 신이 나. [SEP]',
 '[CLS] 당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야. [SEP]',
 '[CLS] 고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워. [SEP]',
 '[CLS] 재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. [SEP]',
 '[CLS] 빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. [SEP]',
 '[CLS] 이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를 하고. 그러면 둘이 하루 종일 함께 있겠네. 생각만 해도 숨이 막혀. [SEP]',
 '[CLS] 사십 대에 접어들면서 머리카락이 많이 빠져 고민이야. [SEP]',
 '[CLS] 이제 돈이라면 지긋지긋해. [SEP]',
 '[CLS] 친구들이 나를 괴롭혀. 부모님과 선생님께 얘기했는데도 믿어주지 않아. [SEP]',
 '[CLS] 친구 때문에 눈물 나. [SEP]']

In [24]:
# 라벨 추출
labels = train['label'].values
labels

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [25]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 아내가 드디어 출산하게 되어서 정말 신이 나. [SEP]
['[CLS]', '아', '##내', '##가', '드', '##디', '##어', '출', '##산', '##하게', '되어', '##서', '정', '##말', '신', '##이', '나', '.', '[SEP]']


In [26]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101,  9519, 31605, 11287,  9113, 48446, 12965,  9768, 21386,
       17594, 37909, 12424,  9670, 89523,  9487, 10739,  8982,   119,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [27]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [28]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9781,  17196,  61688,   8888,  23811,  90373,   9934,  30842,
         12453,   9495, 119185,  14040,  17360,  48549,    119,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [29]:
# 배치 사이즈
batch_size = 16

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [30]:
# 리뷰 문장 추출
sentences = test['sentence']
sentences[:10]

0                                     요즘 부모님과 많이 부딪혀.
1                            엄마가 결국 집을 나갔어. 너무 너무 슬퍼.
2                      학교에서 한 친구를 괴롭히는 무리에게 그만하라고 했어.
3    이번에 팀장님이 간단한 조사 업무를 부탁하셨는데 내가 잘못 처리했어. 너무 절망적이야.
4                   남편이 이혼할 때 위자료를 주지 않으려고 변호사를 고용했어.
5         친구들과 노후에 대한 이야기를 하다 보니 서로 노후 자금 차이가 상당히 컸어.
6                             직장에서 모함을 받았어. 난 정말 억울해.
7                      요즘 딸에게 뭔가 물어보면 신경질부터 내는데 화가 나.
8              언제까지 대출금을 갚으며 살아야 할까? 포기하고 싶어지고 너무 슬퍼.
9                           이 업무는 하루 만에 끝낼 수 있을 것 같아.
Name: sentence, dtype: object

In [31]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 요즘 부모님과 많이 부딪혀. [SEP]',
 '[CLS] 엄마가 결국 집을 나갔어. 너무 너무 슬퍼. [SEP]',
 '[CLS] 학교에서 한 친구를 괴롭히는 무리에게 그만하라고 했어. [SEP]',
 '[CLS] 이번에 팀장님이 간단한 조사 업무를 부탁하셨는데 내가 잘못 처리했어. 너무 절망적이야. [SEP]',
 '[CLS] 남편이 이혼할 때 위자료를 주지 않으려고 변호사를 고용했어. [SEP]',
 '[CLS] 친구들과 노후에 대한 이야기를 하다 보니 서로 노후 자금 차이가 상당히 컸어. [SEP]',
 '[CLS] 직장에서 모함을 받았어. 난 정말 억울해. [SEP]',
 '[CLS] 요즘 딸에게 뭔가 물어보면 신경질부터 내는데 화가 나. [SEP]',
 '[CLS] 언제까지 대출금을 갚으며 살아야 할까? 포기하고 싶어지고 너무 슬퍼. [SEP]',
 '[CLS] 이 업무는 하루 만에 끝낼 수 있을 것 같아. [SEP]']

In [32]:
# 라벨 추출
labels = test['label'].values
labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [33]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 요즘 부모님과 많이 부딪혀. [SEP]
['[CLS]', '요', '##즘', '부', '##모', '##님', '##과', '많이', '부', '##딪', '##혀', '.', '[SEP]']


In [34]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9599, 119229,   9365,  39420, 108578,  11882,  47058,
         9365, 118824,  80579,    119,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [35]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [36]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9599, 119229,   9365,  39420, 108578,  11882,  47058,   9365,
        118824,  80579,    119,    102,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [37]:
# 배치 사이즈
batch_size = 16

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [38]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [39]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1080 Ti


In [40]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [41]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [42]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [43]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [44]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
torch.backends.cudnn.enabled = False

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
       
        # Forward 수행                
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Batch   500  of  12,852.    Elapsed: 0:01:49.
  Batch 1,000  of  12,852.    Elapsed: 0:03:38.
  Batch 1,500  of  12,852.    Elapsed: 0:05:27.
  Batch 2,000  of  12,852.    Elapsed: 0:07:15.
  Batch 2,500  of  12,852.    Elapsed: 0:09:04.
  Batch 3,000  of  12,852.    Elapsed: 0:10:53.
  Batch 3,500  of  12,852.    Elapsed: 0:12:42.
  Batch 4,000  of  12,852.    Elapsed: 0:14:31.
  Batch 4,500  of  12,852.    Elapsed: 0:16:20.
  Batch 5,000  of  12,852.    Elapsed: 0:18:09.
  Batch 5,500  of  12,852.    Elapsed: 0:19:57.
  Batch 6,000  of  12,852.    Elapsed: 0:21:46.
  Batch 6,500  of  12,852.    Elapsed: 0:23:35.
  Batch 7,000  of  12,852.    Elapsed: 0:25:24.
  Batch 7,500  of  12,852.    Elapsed: 0:27:13.
  Batch 8,000  of  12,852.    Elapsed: 0:29:02.
  Batch 8,500  of  12,852.    Elapsed: 0:30:50.
  Batch 9,000  of  12,852.    Elapsed: 0:32:39.
  Batch 9,500  of  12,852.    Elapsed: 0:34:28.
  Batch 10,000  of  12,852.    Elapsed: 0:36:17.
  Batch 10,500  of  12,852.    Elapsed:

C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Accuracy: 0.92
  Validation took: 0:01:21

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  12,852.    Elapsed: 0:01:49.
  Batch 1,000  of  12,852.    Elapsed: 0:03:38.
  Batch 1,500  of  12,852.    Elapsed: 0:05:27.
  Batch 2,000  of  12,852.    Elapsed: 0:07:15.
  Batch 2,500  of  12,852.    Elapsed: 0:09:04.
  Batch 3,000  of  12,852.    Elapsed: 0:10:53.
  Batch 3,500  of  12,852.    Elapsed: 0:12:42.
  Batch 4,000  of  12,852.    Elapsed: 0:14:30.
  Batch 4,500  of  12,852.    Elapsed: 0:16:19.
  Batch 5,000  of  12,852.    Elapsed: 0:18:08.
  Batch 5,500  of  12,852.    Elapsed: 0:19:57.
  Batch 6,000  of  12,852.    Elapsed: 0:21:45.
  Batch 6,500  of  12,852.    Elapsed: 0:23:34.
  Batch 7,000  of  12,852.    Elapsed: 0:25:23.
  Batch 7,500  of  12,852.    Elapsed: 0:27:12.
  Batch 8,000  of  12,852.    Elapsed: 0:29:00.
  Batch 8,500  of  12,852.    Elapsed: 0:30:49.
  Batch 9,000  of  12,852.    Elapsed: 0:32:38.
  Batch 9,500  of  12,852.    Elapsed: 0:34:27.
 

In [45]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(),
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  Batch   100  of  1,785.    Elapsed: 0:00:06.
  Batch   200  of  1,785.    Elapsed: 0:00:11.
  Batch   300  of  1,785.    Elapsed: 0:00:17.
  Batch   400  of  1,785.    Elapsed: 0:00:23.
  Batch   500  of  1,785.    Elapsed: 0:00:28.
  Batch   600  of  1,785.    Elapsed: 0:00:34.
  Batch   700  of  1,785.    Elapsed: 0:00:40.
  Batch   800  of  1,785.    Elapsed: 0:00:45.
  Batch   900  of  1,785.    Elapsed: 0:00:51.
  Batch 1,000  of  1,785.    Elapsed: 0:00:57.
  Batch 1,100  of  1,785.    Elapsed: 0:01:02.
  Batch 1,200  of  1,785.    Elapsed: 0:01:08.
  Batch 1,300  of  1,785.    Elapsed: 0:01:14.
  Batch 1,400  of  1,785.    Elapsed: 0:01:19.
  Batch 1,500  of  1,785.    Elapsed: 0:01:25.
  Batch 1,600  of  1,785.    Elapsed: 0:01:31.
  Batch 1,700  of  1,785.    Elapsed: 0:01:37.

Accuracy: 0.92
Test took: 0:01:41


In [46]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [47]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(torch.tensor(b_input_ids).to(device).long(), 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [48]:
logits = test_sentences(['호잉이 이 감독의 기대에 부응할 때, KT도 더 높은 곳으로 비상할 수 있다.'])

print(logits)
print(np.argmax(logits))

[[-0.97584164  0.6854355 ]]
1


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [49]:
logits = test_sentences(['뒷문이 흔들린 롯데는 전반기를 32승1무44패(승률 0.421)로 마쳤다.'])

print(logits)
print(np.argmax(logits))

[[ 2.374845 -2.615273]]
0


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [66]:
logits = test_sentences(['그러나 로맥은 언제 찾아올지 모를 기회에 대비해 꾸준히 준비했다. '])

print(logits)
print(np.argmax(logits))

[[-1.6330535  1.5442894]]
1


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [57]:
logits = test_sentences(['절호의 찬스 놓친 삼성 원태인'])

print(logits)
print(np.argmax(logits))

[[ 0.90503025 -0.7276854 ]]
0


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [52]:
logits = test_sentences(['투구 맞은 이원석, 교체 후 병원 후송 "팔꿈치 정밀검진 예정"'])

print(logits)
print(np.argmax(logits))

[[ 1.9183775 -2.0392318]]
0


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [56]:
logits = test_sentences(['8일 인천 LG전서 좌월 역전 결승 투런포를 뽑아냈다.'])

print(logits)
print(np.argmax(logits))

[[-1.3128585  1.0872172]]
1


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [54]:
logits = test_sentences(['올 시즌 극심한 부진으로 후반기 초반 2군에 다녀왔고, 1군에 돌아오니 최항과 최주환의 맹활약으로 출전기회가 사실상 막혔다. 대타와 지명타자 등을 오가며 불규칙으로 나가다 보니 더더욱 타격감을 찾기 어려웠다.'])

print(logits)
print(np.argmax(logits))

[[ 2.0438426 -2.2086687]]
0


C:\Users\DeepLearning_1\anaconda3\envs\windup\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [72]:
model.save_pretrained('posnega_model')

In [73]:
tokenizer.save_pretrained('posnega_model')

('posnega_model\\tokenizer_config.json',
 'posnega_model\\special_tokens_map.json',
 'posnega_model\\vocab.txt',
 'posnega_model\\added_tokens.json')